<a href="https://colab.research.google.com/github/nyanta012/demo/blob/main/langchain_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain==0.0.125 openai==0.27.2 tiktoken==0.3.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from langchain.document_loaders.image import UnstructuredImageLoader
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI
import os

os.environ["OPENAI_API_KEY"] = "***"

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=1000)
text_splitter = CharacterTextSplitter(chunk_size=1000)

In [ ]:
with open("ドラゴンボール.txt") as f:
    state_of_the_union = f.read()
texts = text_splitter.split_text(state_of_the_union)

docs = [Document(page_content=t) for t in texts]

In [ ]:
# stuff
%%time
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(docs)

InvalidRequestError: ignored

In [ ]:
# map_reduce
%%time
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(docs)

CPU times: user 281 ms, sys: 33.4 ms, total: 315 ms
Wall time: 52.8 s


"Dragon Ball Z follows Goku as he collects the Dragon Balls, competes in martial arts tournaments, fights against powerful enemies, and saves the world with the help of his friends. The series involves intense battles, the use of the Dragon Balls to revive fallen friends, and ends with Goku leaving to train with Buu's reincarnation, Uub."

In [ ]:
# 要約生成過程を可視化

In [ ]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(docs)



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:
Write a concise summary of the following:


"孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。

その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。

ピッコロ大魔王編
天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。闘いの後、悟空はピッコロ大魔王に殺された神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。そこで神龍復活の条件として、神の下で天界で修行することとなった。

その約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しそ

"Dragon Ball Z follows Goku as he collects the Dragon Balls, competes in martial arts tournaments, fights against powerful enemies, and saves the world. He learns he is a Saiyan and fights against his own brother and other powerful Saiyans. Goku defeats the evil emperor Frieza and later sacrifices himself to defeat Cell. He returns to fight the powerful Majin Buu and ultimately destroys him with the help of a massive energy ball. The story ends with Goku leaving to train with Buu's reincarnation, Uub."

In [ ]:
# プロンプトをカスタマイズする

In [ ]:
from langchain import PromptTemplate

template = """
次の文章を日本語で簡潔に要約してください。
文章：{text}
"""

PROMPT = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [ ]:
chain = load_summarize_chain(llm, chain_type="map_reduce", map_prompt=PROMPT, combine_prompt=PROMPT, verbose=True)

In [ ]:
chain.run(docs)



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:

次の文章を日本語で簡潔に要約してください。
文章：孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。

その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。

ピッコロ大魔王編
天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。闘いの後、悟空はピッコロ大魔王に殺された神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。そこで神龍復活の条件として、神の下で天界で修行することとなった。

その約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しその場で結婚。そして武道会に出場していた

'孫悟空はドラゴンボールを集めて願いを叶える神龍を呼び出す旅をし、修行を積んで強くなり、仲間たちを救い勝利する。その後、天界で修行し、天下一武道会で優勝し、チチと結婚する。数年後、サイヤ人のラディッツが現れ、悟空はピッコロと協力して彼を倒す。更に強力なサイヤ人たちが地球に来襲し、悟空たちは勝利し、地球のドラゴンボールを失う。フリーザが復活し、悟空たちと対決するが、悟空が超サイヤ人に覚醒して勝利する。未来から来たトランクスが、3年後に2体の人造人間が現れることを告げ、セルという究極の人造人間が現れ、悟飯が超サイヤ人2に覚醒してセルを撃破する。魔人ブウの封印が解かれ、悟空とベジータが最終決戦に臨み、超特大の元気玉によって魔人ブウは消滅する。10年後、悟空はウーブと共に修行に旅立ち、物語は幕を閉じる。'

In [ ]:
# refine
%%time
chain = load_summarize_chain(llm, chain_type="refine", verbose=True)
chain.run(docs)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。

その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。

ピッコロ大魔王編
天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。闘いの後、悟空はピッコロ大魔王に殺された神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。そこで神龍復活の条件として、神の下で天界で修行することとなった。

その約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出

'The story follows Son Goku and his friends as they collect the seven Dragon Balls and face off against various enemies, including the Red Ribbon Army, Piccolo, and the Saiyan warriors Nappa and Vegeta. They also travel to Namek to revive their fallen comrades and defeat the tyrant Frieza. Later, they face the threat of the androids and the ultimate android, Cell, who declares a martial arts tournament to determine the fate of the Earth. Goku sacrifices himself in the battle against Cell, but his son Gohan ultimately defeats him and saves the planet. The Dragon Balls are used to revive those killed by Cell, but Goku chooses to remain in the afterlife. In the Majin Buu arc, after seven years, Gohan enters the World Martial Arts Tournament as a high school student, and Goku returns to Earth for one day to participate. During the tournament, they learn that the seal on the powerful Majin Buu is about to be broken. Buu defeats Gohan and Vegeta, and Goku returns to the afterlife, leaving th

In [ ]:
prompt_template = """次の文章の簡潔な要約を書いてください:

{text}

簡潔な日本語の要約:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])


refine_template = (
    
    "あなたの仕事は最終的な要約を作ることです\n"
    "途中までの要約があります: {existing_answer}\n"
    "必要に応じて下記の文章を使い、さらに良い要約を作成してください\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "与えられた文章を踏まえて、日本語で要約を改善してください"
    "与えられた文章が有用でない場合、途中までの文章を返してください"
)
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [ ]:
chain = load_summarize_chain(llm, chain_type="refine", question_prompt=PROMPT, refine_prompt=refine_prompt, verbose=True)
chain.run(docs)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
次の文章の簡潔な要約を書いてください:

孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。

その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。

ピッコロ大魔王編
天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。闘いの後、悟空はピッコロ大魔王に殺された神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。そこで神龍復活の条件として、神の下で天界で修行することとなった。

その約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
あなたの仕事は最終的な要約を作ることです
途中までの要約があります: 孫悟空はドラゴンボールを探す旅に出て、ピラフ一味にボールを奪われるが、ウーロンの下らない願いで野望を阻止する。悟空は修行を積んで強くなり、ピッコロ大魔王に仲間たちが殺されるが、悟空は潜在する力を引き出して勝利する。悟空は神に会い、天界で修行することになる。約3年後、悟空はチチと結婚し、マジュニアとの闘いで初の天下一武道会優勝を飾る。その後、宇宙より来襲したサイヤ人たちとの戦いを経て、悟空たちはナメック星でフリーザとの戦いを繰り広げる。悟空たちはドラゴンボールをめぐる三つ巴の攻防の末、ナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。

その後、フリーザとその一味が地球を襲撃し、悟空たちは謎の超サイヤ人・トランクスの助けを借りて撃退する。トランクスは未来からやってきたブルマとベジータの息子であり、悟空に心臓病による死と2体の人造人間の出現を告げる。悟空たちは3年間の修行を経て、人造人間たちと究極の人造人間・セルとの戦いに臨む。悟空はセルとの闘いで命を落とし、悟飯が超サイヤ人2へと覚醒してセルを撃破する。悟空は生き返りを拒否し、あの世に残ることを選ぶ。
必要に応じて下記の文章を使い、さらに良い要約を作成してください
------------
魔人ブウ編
セルゲームより約7年後、高校生に成長した悟飯が天下一武道会に出場することを知った悟空は、自らも出場するために占いババの力によって1日だけこの世に戻る。天下一武道会の最中、悟空たちは界王よりもさらに高位の存在である界王神から、恐ろしい力を持つ魔人ブウの封印が解かれようとしていることを知らされる。復活した魔人ブウにより悟飯やベジータが倒され、悟空はあの世に帰ったため、地球の命運は悟空の次男・孫悟天と少年トランクスの幼い二人に託される。

一方、魔人ブウは様々な人間との出会いからより邪悪で強力な魔人へと変貌。悟天とトランクスが「フュージョン（融合）」して誕生した戦士・ゴテンクスや、潜在能力を解放し、パワーアップを遂げて帰ってきた悟飯らが応戦するが、戦士たちを次々と吸収し姿


> Finished chain.

> Finished chain.


'ドラゴンボールシリーズの要約は、孫悟空がドラゴンボールを探す旅に出て、多くの敵と戦いながら成長していく物語です。彼はピラフ一味やピッコロ大魔王、フリーザ、セル、そして魔人ブウと戦い、仲間たちと協力して彼らを倒します。最終的に、悟空はウーブと出会い、彼と共に修行に旅立ちます。物語は、悟空が見果てぬ強さを追い求める旅の終わりで幕を閉じます。'